### 1. 安装HuggingFace 并下载模型到本地

In [1]:
!pip install huggingface-hub -Uqq
!pip install -Uqq sagemaker

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
aiobotocore 2.7.0 requires botocore<1.31.65,>=1.31.16, but you have botocore 1.34.34 which is incompatible.


### 如果是外海region，执行下面cell通过huggingface_hub下载, 否则跳过

In [2]:
from huggingface_hub import snapshot_download
from pathlib import Path

local_model_path = Path("./bge-m3-model")
local_model_path.mkdir(exist_ok=True)
model_name = "BAAI/bge-m3"
commit_hash = "4277867103fc67328e2033176de4387b85e9960f"
snapshot_download(repo_id=model_name, revision=commit_hash, cache_dir=local_model_path)

Fetching 19 files:   0%|          | 0/19 [00:00<?, ?it/s]

.gitattributes:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.7k [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/1.31k [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.27G [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

'bge-m3-model/models--BAAI--bge-m3/snapshots/4277867103fc67328e2033176de4387b85e9960f'

### 如果是中国区，执行下面cell通过modelscope进行下载

In [ ]:
!pip install modelscope -i https://pypi.tuna.tsinghua.edu.cn/simple -Uqq

In [ ]:
from modelscope.hub.snapshot_download import snapshot_download
from pathlib import Path

local_model_path = Path("./bge-zh-model")

local_model_path.mkdir(exist_ok=True)
model_name = "Xorbits/bge-large-zh-v1.5"
commit_hash = "v0.0.1"

snapshot_download(model_name, revision=commit_hash, cache_dir=local_model_path)

### 2. 把模型拷贝到S3为后续部署做准备

In [3]:
import sagemaker
from sagemaker import image_uris
import boto3
import os
import time
import json

role = sagemaker.get_execution_role()  # execution role for the endpoint
sess = sagemaker.session.Session()  # sagemaker session for interacting with different AWS APIs
bucket = sess.default_bucket()  # bucket to house artifacts

region = sess._region_name
account_id = sess.account_id()

s3_client = boto3.client("s3")
sm_client = boto3.client("sagemaker")
smr_client = boto3.client("sagemaker-runtime")

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [4]:
s3_model_prefix = "LLM-RAG/workshop/bge-m3-model"  # folder where model checkpoint will go
if region in ['cn-north-1', 'cn-northwest-1']:
    model_snapshot_path = f'{local_model_path}/{model_name}'
else:
    model_snapshot_path = list(local_model_path.glob("**/snapshots/*"))[0]
s3_code_prefix = "LLM-RAG/workshop/bge_m3_deploy_code"
print(f"s3_code_prefix: {s3_code_prefix}")
print(f"model_snapshot_path: {model_snapshot_path}")

s3_code_prefix: LLM-RAG/workshop/bge_m3_deploy_code
model_snapshot_path: bge-m3-model/models--BAAI--bge-m3/snapshots/4277867103fc67328e2033176de4387b85e9960f


In [5]:
!aws s3 cp --recursive {model_snapshot_path} s3://{bucket}/{s3_model_prefix}

upload: bge-m3-model/models--BAAI--bge-m3/snapshots/4277867103fc67328e2033176de4387b85e9960f/config_sentence_transformers.json to s3://sagemaker-us-east-1-631023274615/LLM-RAG/workshop/bge-m3-model/config_sentence_transformers.json
upload: bge-m3-model/models--BAAI--bge-m3/snapshots/4277867103fc67328e2033176de4387b85e9960f/1_Pooling/config.json to s3://sagemaker-us-east-1-631023274615/LLM-RAG/workshop/bge-m3-model/1_Pooling/config.json
upload: bge-m3-model/models--BAAI--bge-m3/snapshots/4277867103fc67328e2033176de4387b85e9960f/imgs/long.jpg to s3://sagemaker-us-east-1-631023274615/LLM-RAG/workshop/bge-m3-model/imgs/long.jpg
upload: bge-m3-model/models--BAAI--bge-m3/snapshots/4277867103fc67328e2033176de4387b85e9960f/colbert_linear.pt to s3://sagemaker-us-east-1-631023274615/LLM-RAG/workshop/bge-m3-model/colbert_linear.pt
upload: bge-m3-model/models--BAAI--bge-m3/snapshots/4277867103fc67328e2033176de4387b85e9960f/config.json to s3://sagemaker-us-east-1-631023274615/LLM-RAG/workshop/bge-m

### 3. 模型部署准备（entrypoint脚本，容器镜像，服务配置）

In [6]:
inference_image_uri = (
    f"763104351884.dkr.ecr.{region}.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118"
    
)

#中国区需要替换为下面的image_uri
if region in ['cn-north-1', 'cn-northwest-1']:
    inference_image_uri = (
        f"727897471807.dkr.ecr.{region}.amazonaws.com.cn/djl-inference:0.23.0-deepspeed0.9.5-cu118"
    )

print(f"Image going to be used is ---- > {inference_image_uri}")

Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118


In [7]:
!mkdir -p bge_m3_deploy_code

In [8]:
%%writefile bge_m3_deploy_code/model.py
from djl_python import Input, Output
import torch
import logging
import math
import os
from FlagEmbedding import BGEM3FlagModel

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f'--device={device}')

def load_model(properties):
    tensor_parallel = properties["tensor_parallel_degree"]
    model_location = properties['model_dir']
    if "model_id" in properties:
        model_location = properties['model_id']
    logging.info(f"Loading model in {model_location}")

    model =  BGEM3FlagModel(model_location,use_fp16=True)
    
    return model

model = None

def handle(inputs: Input):
    global model
    if not model:
        model = load_model(inputs.get_properties())

    if inputs.is_empty():
        return None
    data = inputs.get_as_json()
    
    input_sentences = None
    inputs = data["inputs"]
    if isinstance(inputs, list):
        input_sentences = inputs
    else:
        input_sentences =  [inputs]
        
    is_query = data.get("is_query")
    max_length = data.get("max_length",2048)
    instruction = data.get("instruction")
    logging.info(f"inputs: {input_sentences}")
    logging.info(f"is_query: {is_query}")
    logging.info(f"instruction: {instruction}")
    
    if is_query and instruction:
        input_sentences = [ instruction + sent for sent in input_sentences ]
        
    sentence_embeddings =  model.encode(input_sentences,max_length=max_length)
        
    result = {"sentence_embeddings": sentence_embeddings['dense_vecs']}
    return Output().add_as_json(result)

Writing bge_m3_deploy_code/model.py


In [9]:
print(f"option.s3url ==> s3://{bucket}/{s3_model_prefix}/")

option.s3url ==> s3://sagemaker-us-east-1-631023274615/LLM-RAG/workshop/bge-m3-model/


#### Note: option.s3url 为SageMaker部署时使用的模型S3路径

In [10]:
import os

if not os.path.exists("bge_m3_deploy_code"):
    os.mkdir("bge_m3_deploy_code")

with open('bge_m3_deploy_code/serving.properties', 'w') as f:
    f.write("engine=Python")
    f.write("\n")
    f.write("option.tensor_parallel_degree=1")
    f.write("\n")
    f.write(f"option.s3url=s3://{bucket}/{s3_model_prefix}/")

### 【注意】下面这两个Cell，根据region，仅挑选一个执行，否则会产生问题

#### 如果是中国region，执行下面这个cell，在requirements.txt中添加国内的pip镜像

In [28]:
%%writefile bge_m3_deploy_code/requirements.txt
-i https://pypi.tuna.tsinghua.edu.cn/simple
FlagEmbedding

Overwriting bge_m3_deploy_code/requirements.txt


#### 如果是海外region，执行下面这个cell，无需额外添加pip镜像

In [29]:
%%writefile bge_m3_deploy_code/requirements.txt
FlagEmbedding

Overwriting bge_m3_deploy_code/requirements.txt


In [30]:
!rm s2e_model.tar.gz
!cd bge_m3_deploy_code && rm -rf ".ipynb_checkpoints"
!tar czvf s2e_model.tar.gz bge_m3_deploy_code

bge_m3_deploy_code/
bge_m3_deploy_code/model.py
bge_m3_deploy_code/serving.properties
bge_m3_deploy_code/requirements.txt


In [31]:
s3_code_artifact = sess.upload_data("s2e_model.tar.gz", bucket, s3_code_prefix)
print(f"S3 Code or Model tar ball uploaded to --- > {s3_code_artifact}")

S3 Code or Model tar ball uploaded to --- > s3://sagemaker-us-east-1-631023274615/LLM-RAG/workshop/bge_m3_deploy_code/s2e_model.tar.gz


### 4. 创建模型 & 创建endpoint

In [32]:
from sagemaker.utils import name_from_base
import boto3

model_name = name_from_base("bge-m3") #Note: Need to specify model_name
print(model_name)
print(f"Image going to be used is ---- > {inference_image_uri}")

create_model_response = sm_client.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        "Image": inference_image_uri,
        "ModelDataUrl": s3_code_artifact
    },
    
)
model_arn = create_model_response["ModelArn"]

print(f"Created Model: {model_arn}")

bge-m3-2024-02-04-06-40-11-331
Image going to be used is ---- > 763104351884.dkr.ecr.us-east-1.amazonaws.com/djl-inference:0.23.0-deepspeed0.9.5-cu118
Created Model: arn:aws:sagemaker:us-east-1:631023274615:model/bge-m3-2024-02-04-06-40-11-331


In [33]:
endpoint_config_name = f"{model_name}-config"
endpoint_name = f"{model_name}-endpoint"

endpoint_config_response = sm_client.create_endpoint_config(
    EndpointConfigName=endpoint_config_name,
    ProductionVariants=[
        {
            "VariantName": "variant1",
            "ModelName": model_name,
            "InstanceType": "ml.g4dn.xlarge",
            "InitialInstanceCount": 1,
            # "VolumeSizeInGB" : 400,
            # "ModelDataDownloadTimeoutInSeconds": 2400,
            "ContainerStartupHealthCheckTimeoutInSeconds": 5*60,
        },
    ],
)
endpoint_config_response

{'EndpointConfigArn': 'arn:aws:sagemaker:us-east-1:631023274615:endpoint-config/bge-m3-2024-02-04-06-40-11-331-config',
 'ResponseMetadata': {'RequestId': '2a1c5952-63a4-4cd5-a0de-6bea958f96e7',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '2a1c5952-63a4-4cd5-a0de-6bea958f96e7',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '118',
   'date': 'Sun, 04 Feb 2024 06:40:12 GMT'},
  'RetryAttempts': 0}}

In [34]:
create_endpoint_response = sm_client.create_endpoint(
    EndpointName=f"{endpoint_name}", EndpointConfigName=endpoint_config_name
)
print(f"Created Endpoint: {create_endpoint_response['EndpointArn']}")

Created Endpoint: arn:aws:sagemaker:us-east-1:631023274615:endpoint/bge-m3-2024-02-04-06-40-11-331-endpoint


In [35]:
import time

resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
status = resp["EndpointStatus"]
print("Status: " + status)

while status == "Creating":
    time.sleep(60)
    resp = sm_client.describe_endpoint(EndpointName=endpoint_name)
    status = resp["EndpointStatus"]
    print("Status: " + status)

print("Arn: " + resp["EndpointArn"])
print("Status: " + status)

Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: Creating
Status: InService
Arn: arn:aws:sagemaker:us-east-1:631023274615:endpoint/bge-m3-2024-02-04-06-40-11-331-endpoint
Status: InService


### 5. 模型测试

In [36]:
def get_vector_by_sm_endpoint(questions, sm_client, endpoint_name):
    parameters = {
    }

    response_model = sm_client.invoke_endpoint(
        EndpointName=endpoint_name,
        Body=json.dumps(
            {
                "inputs": questions,
                # "is_query": True,
                "instruction" :  "Represent this sentence for searching relevant passages:"
            }
        ),
        ContentType="application/json",
    )
    # 中文instruction => 为这个句子生成表示以用于检索相关文章：
    json_str = response_model['Body'].read().decode('utf8')
    json_obj = json.loads(json_str)
    embeddings = json_obj['sentence_embeddings']
    return embeddings

In [39]:
prompts1 = ["How to compete with OCI","如何与OCI竞争","如何與OCI競爭"]

emb = get_vector_by_sm_endpoint(prompts1, smr_client, endpoint_name)

In [45]:
import numpy as np
def cos_sim(vector1, vector2):
    dot_product = np.dot(vector1, vector2)
    norm_v1 = np.linalg.norm(vector1)
    norm_v2 = np.linalg.norm(vector2)
    cos_sim = dot_product / (norm_v1 * norm_v2)
    return cos_sim

In [48]:
cos_sim(emb[1],emb[2])

0.9856032083678408

#### 清除模型Endpoint和config （如无需要，不要执行）

In [ ]:
!aws sagemaker delete-endpoint --endpoint-name bge-large-en-2023-08-16-09-58-49-900-endpoint

In [ ]:
!aws sagemaker delete-endpoint-config --endpoint-config-name bge-large-en-2023-08-16-09-58-49-900-config

In [ ]:
!aws sagemaker delete-model --model-name bge-large-en-2023-08-16-09-58-49-900